In [1]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Embedding, MaxPool1D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Bidirectional, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import glorot_uniform, RandomUniform, lecun_uniform, Constant
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPool1D
import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report

# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler

from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.tokenize import word_tokenize
import gensim.downloader
import io
from keras_preprocessing.text import tokenizer_from_json
from gensim.models import Word2Vec, FastText
from sklearn.neighbors import KNeighborsClassifier



D:\anaconda3\envs\tfgpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set the seeder to have as stable random operations as possible

In [2]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [3]:
embedding_algorithms = ["w2v", "ft", "py_w2v", "py_ft"]
embedding_algorithm = embedding_algorithms[3]

In [4]:
representation_form = "sequences"

In [5]:
# binary or multi-class
multi = True

Read data

In [6]:
if representation_form == "bow":
    data = pd.read_csv('bow_data.csv') # bow
else:
    data = pd.read_csv('sequences_data.csv') # sequences of tokens

In [7]:
data.head()

,Vulnerability,Category,Length
0,"f""str$id""""str$id""""str$id"" ...",sql_injection,9
1,client.listentcp() proxy = proxy(proxy_...,xsrf,8
2,"from django.http import httpresponse, httpresp...",open_redirect,9
3,"def write_preset(conn, queryin, descriptin):\t...",sql_injection,175
4,update_query = self.up...,sql_injection,14


In [8]:
if representation_form == "bow":
    bow_size = 237 # number of columns that stand as features

In [9]:
np.max(data["Length"])

392

In [10]:
if representation_form == "sequences":
    word_counts = data["Vulnerability"].apply(lambda x: len(x.split()))
    max_length = word_counts.max()
    print("Maximum number of words:", max_length)


Maximum number of words: 392


In [11]:
label_frequencies = data['Category'].value_counts()
print("Label Frequencies:\n", label_frequencies)
print("Total samples ", len(data))

Label Frequencies:
 sql_injection            1424
xsrf                      976
command_injection         721
path_disclosure           481
open_redirect             442
remote_code_execution     334
xss                       145
Name: Category, dtype: int64
Total samples  4523


In [12]:
if multi == True:
    n_categories = len(label_frequencies) # 7
    print(n_categories)

7


In [13]:
if multi == True:
    
    # Convert categories to numerical indexes
    category_numerical_indexes, unique_categories = data["Category"].factorize()

    # Create a dictionary mapping each category to its numerical index
    category_to_index = {category: index for index, category in enumerate(unique_categories)}

    # Update the categories in the DataFrame with their numerical indexes
    data["Category_Index"] = data["Category"].map(category_to_index)


Word Embedding

Word2Vec - load pre-trained word2vec embeddings - NL knowledge - static embeddings

In [14]:
if representation_form == "sequences":

    # Download the Punkt tokenizer models if not already downloaded
    nltk.download('punkt')

    tokenized_list = [word_tokenize(sentence) for sentence in data["Vulnerability"].tolist()]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iliaskaloup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BERT - load pre-trained bert embeddings - NL knowledge - contextual embeddings

In [15]:
def vectorize(words, w2v_vectors, dim):
    words_vecs = [w2v_vectors[word] for word in words if word in w2v_vectors]
    if len(words_vecs) == 0:
        return np.zeros(dim)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [16]:
if embedding_algorithm == "w2v":
    vectors = gensim.downloader.load('word2vec-google-news-300')
    dim = 300

In [17]:
if embedding_algorithm == "ft":
    vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')
    dim = 300

In [18]:
if embedding_algorithm == "py_w2v":
    vectors = Word2Vec.load("w2v_model.model").wv
    dim = 100

In [19]:
if embedding_algorithm == "py_ft":
    vectors = FastText.load("fast-text_model.model").wv
    dim = 100

In [20]:
embs = np.array([vectorize(sentence, vectors, dim) for sentence in np.array(tokenized_list)])

D:\anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


Cross-Validation

Multi-class Classification: Categorization of all detected vulnerabilities

In [21]:
def runMLCrossVal_multi(X, y, seed, userModel):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        if userModel == "SVM":
            myModel = SVC(kernel='rbf', gamma=100)
        elif userModel == "RF":
            myModel = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
        elif userModel == "DT":
            myModel = tree.DecisionTreeClassifier(max_depth=120)
        elif userModel == "NB":
            myModel = GaussianNB()
        elif userModel == "KNN":
            myModel = KNeighborsClassifier(n_neighbors=1)
            
        myModel.fit(X_train, Y_train.ravel())
        
        predictions = myModel.predict(X_test)
        #predScores = myModel.predict_proba(X_test)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions, average='macro')
        recall=recall_score(Y_test, predictions, average='macro')
        f1=f1_score(Y_test, predictions, average='macro')
        conf_matrix = confusion_matrix(Y_test, predictions)
        print(conf_matrix)
        #tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
        #acc = ((tp+tn)/(tp+tn+fp+fn))
        print("Accuracy:%.2f%%"%(accuracy*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))


In [22]:
embs = pd.DataFrame(embs)

In [26]:
if multi == True and representation_form == "sequences" and embedding_algorithm == "w2v":
    runMLCrossVal_multi(embs, data["Category_Index"], seed, "KNN")
elif multi == True and representation_form == "sequences" and embedding_algorithm == "ft":
    runMLCrossVal_multi(embs, data["Category_Index"], seed, "KNN")
elif multi == True and representation_form == "sequences" and embedding_algorithm == "py_w2v":
    runMLCrossVal_multi(embs, data["Category_Index"], seed, "KNN")
elif multi == True and representation_form == "sequences" and embedding_algorithm == "py_ft":
    runMLCrossVal_multi(embs, data["Category_Index"], seed, "KNN")

Training...
fold number=  1
[[106   8  10   6   2   5   6]
 [  5  85   2   0   1   4   0]
 [  8   2  29   2   2   0   2]
 [  1   2   2   9   0   0   0]
 [  4   1   0   1  26   2   0]
 [  8   1   4   2   3  52   2]
 [  3   4   2   0   2   2  35]]
Accuracy:75.50%
Precision:70.75%
Recall:73.16%
F1 score:71.62%
              precision    recall  f1-score   support

           0       0.79      0.74      0.76       143
           1       0.83      0.88      0.85        97
           2       0.59      0.64      0.62        45
           3       0.45      0.64      0.53        14
           4       0.72      0.76      0.74        34
           5       0.80      0.72      0.76        72
           6       0.78      0.73      0.75        48

    accuracy                           0.75       453
   macro avg       0.71      0.73      0.72       453
weighted avg       0.76      0.75      0.76       453

fold number=  2
[[111  10   5   2   5   5   5]
 [  2  90   1   0   0   3   1]
 [  1   2  36   